# Projekat iz Soft Computing-a

### Detekcija slicnosti crta lica i kreiranje usrednjenog lica

Algoritam za pronalaženje osobe/karakteristika lica koje su najinteresantnije (najčešće pojavljivane).

Ulaz: skup od 5-20 slika lica (samo portret, bez iskrivljena) osoba koje nam se sviđaju (izbor od nekoliko slika ili samostalan izbor)

Izlaz: skup (top 3) karakteristike koje su najbitnije (najčešće se javljaju) kod lica osoba. Dodatno, prikaz lica koje je kreirano na osnovu rezultata tako da najbolje odgovara korisničkom ukusu. Verbositi režim ispisuje sve detalje izlaza iz pojedinačnih procesnih jedinica.

Algoritamski koraci:
- skup ulaznih slika stize na obradu
- slike prolaze predprocesiranje (izjednačenje osvetljenja, iste dimenzije,...)
- izvlače se lica osoba
- izvlači se kosa osoba
- kosa osobe ulazi u deo algoritama koji određuje boju kose (crna, braon, plava, crvena, neka druga boja)
- lice osobe ulazi u deo algoritama za određivanje karakteristični crta lica (reprezentativne tačke lica)
- određuju se karakteristične mere (veličina nosa, rastojanje između očiju, debljina usana, stepen vertikalne simetrije lica, zlatni presek lica,...)
- iz lica se izvlači deo koji predstavlja oči (tačnije zenicu oka) i određuje se boja oka (možda bi zbog heterohromije bilo bolje odrediti boju oba oka nezavisno pa utvrditi postojanje heterohromije)
- deo lica (možda deo direktno ispod oka) ulazi u deo algoritama za određivanje boje kože (svetloputi i tamnoputi)

- na osnovu dobijenih izlaza četri dela algoritma (boja kose, boja kože, boja očiju i karakteristične mere lica) određuje se top tri karakteristike koje su najčešće između osoba sa slike
- (možda) neuronska mreža kreira lice osobe koja je mean (prosek) svih crta koje su bitne (boja kose, boja očiju, boja kože i karakteristične mere lica bi trebalo da igraju ulogu u ovom delu algoritama).


#### Facial landmarks

![title](facial_landmarks.jpeg)

#### Sema algoritma

![title](diagram.png)

### Kodiranje algoritma

#### Ukljucivanje biblioteka

In [1]:
import os
import cv2
import math
import dlib
import operator
import webcolors
import itertools
import statistics
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from colorthief import ColorThief
from imutils import face_utils
from sklearn.svm import SVC

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = 20,16

#### Deklarisanje funkcija

In [2]:
verbosity=False

def load_image(path):
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

def print_image(image,title=''):
    if verbosity==False:
        return
    figure = plt.figure()
    plt.title(title)
    plt.imshow(image)
    
def euclidean_distance(x1,y1,x2,y2):
    return math.hypot(x2 - x1, y2 - y1)

def golden_ratio(shape):
    GR = 1.62
    err_list = [] # sum of square errors
    
    # odnos debljina usne - sirina nosa
    x1 = get_lips_height(shape)
    x2 = get_nose_width(shape)
    err_list.append((GR-x1/x2)**2)
    
    # odnos sirina nosa i razmak usana od nosa
    x1 = get_nose_width(shape)
    mouth_centre = line_centre(shape[62],shape[66])
    x2 = euclidean_distance(shape[33][0],shape[33][1],mouth_centre[0],mouth_centre[1])
    err_list.append((GR-x1/x2)**2)
    
    # odnos razmaka od vrha nosa do brade i razmak usana od vrha brade
    x1 = euclidean_distance(shape[33][0],shape[33][1],shape[8][0],shape[8][1])
    x2 = euclidean_distance(shape[8][0],shape[8][1],mouth_centre[0],mouth_centre[1])
    err_list.append((GR-x1/x2)**2)
    
    # jednak odnos sirine ociju i sirine medjuocnog prostora
    x1 = euclidean_distance(shape[36][0],shape[36][1],shape[39][0],shape[39][1])
    x2 = euclidean_distance(shape[42][0],shape[42][1],shape[45][0],shape[45][1])
    eye_width = (x1+x2)/2
    x3 = get_eye_distance(shape)
    
    err = eye_width/x3
    if (err<0):
        err = x3/eye_width
    
    err_list.append(err**2)
    
    # jednak odnos po visini
    left_eye = line_centre(shape[36],shape[39])
    right_eye = line_centre(shape[42],shape[45])
    eyes = line_centre(left_eye,right_eye)
    
    x1 = euclidean_distance(shape[33][0],shape[33][1],eyes[0],eyes[1])
    x2 = euclidean_distance(shape[33][0],shape[33][1],shape[8][0],shape[8][1])
    
    err = x1/x2
    if (err<0):
        err = x2/x1
    
    err_list.append(err**2)
    
    return sum(err_list)

def get_nose_height(shape):
    shape = shape.tolist()
    high_point = shape[27]
    low_point = shape[33]
    return euclidean_distance(high_point[0],high_point[1],low_point[0],low_point[1])

def get_nose_width(shape):
    shape = shape.tolist()
    high_point = shape[31]
    low_point = shape[35]
    return euclidean_distance(high_point[0],high_point[1],low_point[0],low_point[1])

def get_lips_height(shape):
    shape = shape.tolist()
    high_point = shape[51]
    low_point = shape[57]
    return euclidean_distance(high_point[0],high_point[1],low_point[0],low_point[1])

def get_eybrows_hight(shape):
    shape = shape.tolist()
    left_eybrow = shape[19]
    right_eybrow = shape[24]
    eybrows = line_centre(left_eybrow,right_eybrow)
    chin = shape[8]
    return euclidean_distance(chin[0],chin[1],eybrows[0],eybrows[1])

def get_eye_distance(shape):
    shape = shape.tolist()
    left_point = shape[39]
    right_point = shape[42]
    return euclidean_distance(left_point[0],left_point[1],right_point[0],right_point[1])

def line_centre(x,y):
    return [int(round((x[0]+y[0])/2)),int(round((x[1]+y[1])/2))]

def slope_interception(x_val,y_val):
    x = np.array(x_val)
    y = np.array(y_val)
    m = ( ((np.mean(x)*np.mean(y)) - np.mean(x*y)) / ((np.mean(x)*np.mean(x)) - np.mean(x*x)) )
    m = round(m,2)
    b = (np.mean(y) - np.mean(x)*m)
    b = round(b,2)
    # y = m*x+b
    return m,b

def left_right_symmetry_line(shape):
    shape = shape.tolist()
    points = [ (21,22), (18,25), (36,45), (39,42), (31,35), (48,54), (50,52), (58,56), (5,11), (1,15), (3,13), (7,9) ]
    x_val = []
    y_val = []
    for item in points:
        left_point = shape[item[0]]
        right_point = shape[item[1]]
        (x,y) = line_centre(left_point,right_point)
        x_val.append(x)
        y_val.append(y)
        
    (m,b) = slope_interception(x_val,y_val)
    return m,b,x_val,y_val

def main_symmetry_line(shape):
    shape = shape.tolist()
    points = [27,27,29,30,33,51,62,66,57,8]
    x_val = []
    y_val = []
    for item in points:
        point = shape[item]
        x_val.append(point[0])
        y_val.append(point[1])
        
    (m,b) = slope_interception(x_val,y_val)
    return m,b,x_val,y_val

def symmetry_deviation(shape):
    # calculate symmetry line of shape (left-right lines)
    (m1,b1,x_val,y_val) = left_right_symmetry_line(shape)
    reg_line = [(m1*x)+b1 for x in x_val]
    #err = square_error(y_val,reg_line)

    #print('Simetrija je: ' + str(err) + '%')
    #plt.scatter(x_val,y_val,color="red")
    if verbosity==True:
        plt.plot(x_val,reg_line)

    # main symmetry line of shape (nose-chin line)
    (m2,b2,x_val,y_val) = main_symmetry_line(shape)
    reg_line = [(m2*x)+b2 for x in x_val]
    #err = square_error(y_val,reg_line)
    #plt.scatter(x_val,y_val,color="yellow")
    if verbosity==True:
        plt.plot(x_val,reg_line)
    
    # determin deviation of two lines
    err = abs(math.degrees(math.atan((m1-m2)/(1+m1*m2))))
    return err

def eye_color(shape,face_image):
    (x, y, w, h) = cv2.boundingRect(np.array([shape[42:47]]))
    right_eye = face_image[y:y+h, x:x+w] #[y + 6:y + 10, x + 10:x + 15]
    #print_image(right_eye)

    #cv2.rectangle(right_eye,(int(w/3),int(h/2)),(int((4*w)/7),int((2*h)/3)),(0,255,0),1)
    #print_image(right_eye)
    
    h, w, channels = right_eye.shape
    iris = right_eye[int(h/2):int((2*h)/3),int(w/3):int((4*w)/7)]
    print_image(iris)

    return get_dominant_color(iris)

def skin_color(shape,face_image):
    shape = shape.tolist()
    xmin = shape[42][0]
    ymax = shape[30][1]
    xmax = shape[45][0]
    ymin = shape[28][1]

    #cv2.rectangle(face_image,(xmin,ymin),(xmax,ymax),(0,255,0),3)
    #print_image(face_image)
                
    skin = face_image[ymin:ymax, xmin:xmax]
    print_image(skin)

    return get_dominant_color(skin)

def skin_wrinkles(shape,face_image):
    shape = shape.tolist()
    xmin = shape[42][0]
    ymax = shape[30][1]
    xmax = shape[45][0]
    ymin = shape[28][1]

    #cv2.rectangle(face_image,(xmin,ymin),(xmax,ymax),(0,255,0),3)
    #print_image(face_image)
                
    skin = face_image[ymin:ymax, xmin:xmax]
    gray_skin = cv2.cvtColor(skin, cv2.COLOR_RGB2GRAY)
    
    v = np.median(gray_skin)
    sigma = 0.33    
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(gray_skin, lower, upper)
    
    print_image(edged,'Gray skin')
    
    white_pixels = np.sum(edged == 255)
    width,height = edged.shape
    total_pixels = width*height
    #print('white: ' + str(white_pixels) + ', total: ' + str(total_pixels))
    
    return white_pixels/total_pixels

def hair_color(shape,face_image):
    shape = shape.tolist()
    colorL = get_left_hair_color(shape,face_image)
    colorR = get_right_hair_color(shape,face_image)
    
    medianR = int((colorL[0] + colorR[0])/2)
    medianG = int((colorL[1] + colorR[1])/2)
    medianB = int((colorL[2] + colorR[2])/2)

    return [medianR, medianG, medianB]

def get_right_hair_color(shape,face_image):
    rx=int((shape[16][0]+shape[15][0])/2)
    ry=2*shape[16][1]-shape[15][1]
    dx=dy=5
    H_shift=0
    W_shift=5
    
    xmin = rx-dx+W_shift
    ymax = ry+dy+H_shift
    xmax = rx+dx+W_shift
    ymin = ry-dy+H_shift

    #cv2.rectangle(face_image,(xmin,ymin),(xmax,ymax),(0,255,0),3)
    #print_image(face_image)
                
    hair = face_image[ymin:ymax, xmin:xmax]
    #print_image(hair)

    return get_dominant_color(hair)

def get_left_hair_color(shape,face_image):
    rx=int((shape[0][0]+shape[1][0])/2)
    ry=2*shape[0][1]-shape[1][1]
    dx=dy=5
    H_shift=0
    W_shift=-5
    
    xmin = rx-dx+W_shift
    ymax = ry+dy+H_shift
    xmax = rx+dx+W_shift
    ymin = ry-dy+H_shift

    #cv2.rectangle(face_image,(xmin,ymin),(xmax,ymax),(0,255,0),3)
    #print_image(face_image)
                
    hair = face_image[ymin:ymax, xmin:xmax]
    #print_image(hair)

    return get_dominant_color(hair)
    
def get_dominant_color(image):
    cv2.imwrite('temp.jpg', image)
    color_thief = ColorThief('temp.jpg')
    os.remove("temp.jpg")
    # get the dominant color - BGR
    dominant_color = color_thief.get_color(quality=1)
    lighten_factor = 1.0
    dominant_color = [int(dominant_color[2]*lighten_factor), int(dominant_color[1]*lighten_factor), int(dominant_color[0]*lighten_factor)]
    return dominant_color

def closest_color(requested_color):
    min_colors = {}
    mycolors = { "#A52A2A":"brown", "#008000":"green","#0000FF":"blue", "#00FFFF":"blue",
                "#000000":"black", "#808080":"gray", "#00FF00":"green", "#800000":"brown",
               "#800080":"purple", "#FF0000":"red", "#FFFFFF":"white", "#8D5524":"brown",
               "#D2B48C":"tan", "#FFDBAC":"sand white"}
    # for key, name in webcolors.css3_hex_to_names.items():
    for key, name in mycolors.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_color[0]) ** 2
        gd = (g_c - requested_color[1]) ** 2
        bd = (b_c - requested_color[2]) ** 2
        min_colors[(rd + gd + bd)] = name
    return min_colors[min(min_colors.keys())]

def get_color_name(requested_color):
    try:
        color_name = webcolors.rgb_to_name(requested_color)
    except ValueError:
        color_name = closest_color(requested_color)
    return color_name

def most_common(L):
  # get an iterable of (item, iterable) pairs
  SL = sorted((x, i) for i, x in enumerate(L))
  # print 'SL:', SL
  groups = itertools.groupby(SL, key=operator.itemgetter(0))
  # auxiliary function to get "quality" for an item
  def _auxfun(g):
    item, iterable = g
    count = 0
    min_index = len(L)
    for _, where in iterable:
      count += 1
      min_index = min(min_index, where)
    # print 'item %r, count %r, minind %r' % (item, count, min_index)
    return count, -min_index
  # pick the highest-count/earliest item
  return max(groups, key=_auxfun)[0]

def average(l): 
    return sum(l)/len(l)


#### Implementacija algoritma

In [3]:
# Deklarisanje promenljivih
image_list = {}
list_eye_color = []
list_skin_color = []
list_hair_color = []
list_wrinkles = []
list_symmerty = []
list_golden_ratio = []

pos_features = []
neg_features = []
labels = []

image_dir_pos = 'slike/positive'
image_dir_neg = 'slike/negative'

# Algoritam 1: Preuzimanje svih slika iz foldera
# image_list = [Image.open(item) for i in [glob.glob('slike/*.%s' % ext) for ext in ["jpg","gif","png"]] for item in i]

for img_name in os.listdir(image_dir_pos):
    img_path = os.path.join(image_dir_pos, img_name)
    img = load_image(img_path)
    img_name = "pos_" + img_name
    image_list[img_name]=img
    
for img_name in os.listdir(image_dir_neg):
    img_path = os.path.join(image_dir_neg, img_name)
    img = load_image(img_path)
    img_name = "neg_" + img_name
    image_list[img_name]=img

# Algoritam 2: Predprocesiranje slika
#for img in image_list:
#    image_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#    temp_list.append(image_hsv)

#image_list = temp_list


# Algoritam 3: Face landmarks extraction
p = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

for name in image_list:
    image = image_list[name]
    original = image.copy()
    features = []
    if verbosity==True:
        print('\nSlika na obradi je ' + name)
    
    # Converting the image to gray scale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    rects = detector(gray, 0)

    # For each detected face, find the landmark.
    for (i, rect) in enumerate(rects):
        # Make the prediction and transfom it to numpy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

    # Draw on our image, all the finded cordinate points (x,y) 
    for (x, y) in shape:
        cv2.circle(image, (x, y), 2, (0, 255, 0), -1)        
    
    # Show the image with landmarks
    print_image(image,name)
    
    # Algoritam 4: Symmetry of the face
    err = symmetry_deviation(shape)
    if verbosity==True:
        print('Odstupanje od simetrije je ' + str(err) + '° (manje je bolje, idealno je 0)')
    list_symmerty.append(err)
    features.append(err)
    
    # Algoritam 5: Golden ratio
    err = golden_ratio(shape)
    if verbosity==True:
        print('Odstupanje od Zlatnog preseka je ' + str(err) + ' (manje je bolje, idealno je 1)')
    list_golden_ratio.append(err)
    features.append(err)
    
    # Algoritam 6: Eye color
    dominant_color = eye_color(shape,original)
    color_name = get_color_name(dominant_color)
    if verbosity==True:
        print('Dominantna boja ociju: RGB' + str(dominant_color) + ' naziv te boje je: ' + color_name)
    list_eye_color.append(color_name)
    # features.append(color_name)
    
    # Algoritam 7: Skin color
    dominant_color = skin_color(shape,original)
    color_name = get_color_name(dominant_color)
    if verbosity==True:
        print('Dominantna boja koze: RGB' + str(dominant_color) + ' naziv te boje je: ' + color_name)
    list_skin_color.append(color_name)
    # features.append(color_name)
    
    # Algoritam 8: Hair color
    dominant_color = hair_color(shape,original)
    color_name = get_color_name(dominant_color)
    if verbosity==True:
        print('Dominantna boja kose: RGB' + str(dominant_color) + ' naziv te boje je: ' + color_name)
    list_hair_color.append(color_name)
    # features.append(color_name)
    
    # Algoritam 9: Skin wrinkles
    wrinkles_number = skin_wrinkles(shape,original)
    if verbosity==True:
        print('Procenat bora je ' + str(wrinkles_number) + ' (manje je bolje, idealno je 0)')
    list_wrinkles.append(wrinkles_number)
    features.append(wrinkles_number)
    
    # Algoritam 10: Priprema za klasifikaciju
    img_type = name.split("_", 1)[0]
    if (img_type=="pos"):
        pos_features.append(features)
        labels.append(1)
    else:
        neg_features.append(features)
        labels.append(0)
    

print('\nRezultati:')
print('Prosecno odstupanje od simetrije:        ' + str(average(list_symmerty)) + '° (manje je bolje, idealno je 0)')
print('Prosecno odstupanje od zlatnog preseka:  ' + str(average(list_golden_ratio)) + ' (manje je bolje, idealno je 1)')
print('Najcesci procenat bora:  ' + str(average(list_wrinkles)) + '% (manje je bolje, idealno je 0)')
print('Najcesca boja ociju: ' + most_common(list_eye_color))
print('Najcesca boja koze:  ' + most_common(list_skin_color))
print('Najcesca boja kose:  ' + most_common(list_hair_color))



Rezultati:
Prosecno odstupanje od simetrije:        5.164449113172806° (manje je bolje, idealno je 0)
Prosecno odstupanje od zlatnog preseka:  2.341954434734231 (manje je bolje, idealno je 1)
Najcesci procenat bora:  0.055893701549290284% (manje je bolje, idealno je 0)
Najcesca boja ociju: black
Najcesca boja koze:  tan
Najcesca boja kose:  black


In [4]:
# Algoritam 10: Priprema za klasifikaciju
pos_features = np.array(pos_features)
neg_features = np.array(neg_features)
x = np.vstack((pos_features, neg_features))
y = np.array(labels)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [5]:
#  Algoritam 11: SVM klasifikacija
svm = SVC(kernel='linear', probability=True) 
svm.fit(x_train, y_train)
y_train_pred = svm.predict(x_train)
y_test_pred = svm.predict(x_test)
print("Train accuracy: ", accuracy_score(y_train, y_train_pred))
print("Validation accuracy: ", accuracy_score(y_test, y_test_pred))

Train accuracy:  0.6481481481481481
Validation accuracy:  0.6428571428571429


In [6]:
# Algoritam 12: KNN klasifikacija
knn = KNeighborsClassifier(n_neighbors=10)
knn = knn.fit(x_train, y_train)
y_train_pred = knn.predict(x_train)
y_test_pred = knn.predict(x_test)
print("Train accuracy: ", accuracy_score(y_train, y_train_pred))
print("Validation accuracy: ", accuracy_score(y_test, y_test_pred))

Train accuracy:  0.5370370370370371
Validation accuracy:  0.7142857142857143


In [7]:
import os
import cv2
import math
import dlib
import operator
import datetime
import itertools
import statistics
import matplotlib
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

def load_image(path):
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

def reshape_data(input_data):
    nsamples, nx, ny = input_data.shape
    return input_data.reshape((nsamples, nx*ny))


image_list = {}

pos_features = []
neg_features = []
labels = []

image_dir_pos = 'slike/positive'
image_dir_neg = 'slike/negative'

nbins = 9 # broj binova (unutar histograma ima 9 elemenata)
cell_size = (8, 8) # broj piksela po celiji
block_size = (3, 3) # broj celija po bloku

pos_imgs = []
neg_imgs = []

print('\n*** Algoritam koji ubacuje HOG slike u SVM i KNN za \'glupu\' detekciju narkomana ***')
print('\nProcess started at ', datetime.datetime.utcnow(),'\n')

# Algoritam: Preuzimanje svih slika iz foldera
for img_name in os.listdir(image_dir_pos):
    img_path = os.path.join(image_dir_pos, img_name)
    img = load_image(img_path)
    img = cv2.resize(img,(50,25))
    pos_imgs.append(img)

for img_name in os.listdir(image_dir_neg):
    img_path = os.path.join(image_dir_neg, img_name)
    img = load_image(img_path)
    img = cv2.resize(img,(50,25))
    neg_imgs.append(img)

print("Positive images: ", len(pos_imgs))
print("Negative images: ", len(neg_imgs))


hog = cv2.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1], 
                                  img.shape[0] // cell_size[0] * cell_size[0]),
                        _blockSize=(block_size[1] * cell_size[1],
                                    block_size[0] * cell_size[0]),
                        _blockStride=(cell_size[1], cell_size[0]),
                        _cellSize=(cell_size[1], cell_size[0]),
                        _nbins=nbins)


# Algoritam: Izracunavanje HOG-a slika i punjenje lista
for img in pos_imgs:
    pos_features.append(hog.compute(img))
    labels.append(1)

for img in neg_imgs:
    neg_features.append(hog.compute(img))
    labels.append(0)

# Algoritam: Priprema za klasifikaciju
pos_features = np.array(pos_features)
neg_features = np.array(neg_features)
x = np.vstack((pos_features, neg_features))
y = np.array(labels)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train = reshape_data(x_train)
x_test = reshape_data(x_test)

#  Algoritam: SVM klasifikacija
svm = SVC(kernel='linear', probability=True) 
svm.fit(x_train, y_train)
y_train_pred = svm.predict(x_train)
y_test_pred = svm.predict(x_test)
print("SVM Train accuracy: ", accuracy_score(y_train, y_train_pred))
print("SVM Validation accuracy: ", accuracy_score(y_test, y_test_pred))

# Algoritam: KNN klasifikacija
knn = KNeighborsClassifier(n_neighbors=10)
knn = knn.fit(x_train, y_train)
y_train_pred = knn.predict(x_train)
y_test_pred = knn.predict(x_test)
print("KNN Train accuracy: ", accuracy_score(y_train, y_train_pred))
print("KNN Validation accuracy: ", accuracy_score(y_test, y_test_pred))


print('\nProcess ended at ', datetime.datetime.utcnow())


*** Algoritam koji ubacuje HOG slike u SVM i KNN za 'glupu' detekciju narkomana ***

Process started at  2019-01-17 19:39:19.334443 

Positive images:  34
Negative images:  34
SVM Train accuracy:  0.7777777777777778
SVM Validation accuracy:  0.5714285714285714
KNN Train accuracy:  0.6481481481481481
KNN Validation accuracy:  0.42857142857142855

Process ended at  2019-01-17 19:39:19.956827
